# YOLO形式データセットの作成（画像+ラベル1対1対応）

このNotebookでは、既に生成されたラベルファイルに対応する画像ファイルをコピーし、
YOLO形式のデータセット（画像とラベルの1対1対応）を作成します。

## 処理の流れ
1. Google Driveのマウント
2. Gitリポジトリのクローン/更新
3. 環境変数の設定
4. 画像マッピングファイルの読み込み
5. 画像ファイルのコピー（`processed/{split}/images/{uuid}.png`）
6. 画像とラベルの対応確認・可視化

## 使用方法について

このNotebookは`preprocess_kaggle_data.ipynb`の後に実行してください。

- **前提**: `preprocess_kaggle_data.ipynb`でラベルファイルと`image_mapping.json`が生成されていること
- **出力**: `processed/{split}/images/{uuid}.png` に画像ファイルをコピー
- **結果**: `images/{uuid}.png` ↔ `labels/{uuid}.txt` の1対1対応が完成

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!git clone -b feature/onnx_recognizer https://github.com/masararutaru/last_assignment_progzissen.git
# %cd last_assignment_progzissen
# !git pull  # 毎回実行

# 環境変数の設定
import os
os.environ["DATA_ROOT"] = "/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment"
os.environ["OUT_ROOT"] = "/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment"

print("環境変数設定完了")
print(f"DATA_ROOT: {os.environ.get('DATA_ROOT')}")
print(f"OUT_ROOT: {os.environ.get('OUT_ROOT')}")

In [ ]:
# パスの設定と確認
import sys
import json
import shutil
from pathlib import Path
from tqdm import tqdm

# パスを追加（リポジトリのpython/dataディレクトリ）
repo_path = Path('/content/last_assignment_progzissen')
sys.path.insert(0, str(repo_path / 'python' / 'data'))

from config import CLASSES, CLASS_TO_ID

# 環境変数からパスを取得（必須チェック）
out_root = os.environ.get("OUT_ROOT")
if not out_root:
    raise ValueError(
        "❌ エラー: OUT_ROOT環境変数が設定されていません。\n"
        "   セル2で環境変数を設定してください。\n"
        "   例: os.environ['OUT_ROOT'] = '/content/drive/MyDrive/...'"
    )

# パスの検証（MyDrive直下に保存されないようにする）
out_root_path = Path(out_root)
if str(out_root_path) == "/content/drive/MyDrive" or str(out_root_path.parent) == "/content/drive/MyDrive":
    raise ValueError(
        f"❌ エラー: OUT_ROOTがMyDrive直下を指しています: {out_root}\n"
        "   これは許可されていません。必ずサブディレクトリを指定してください。\n"
        "   例: /content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment"
    )

split = 'train'  # 'train', 'val', 'test' を変更可能

# パスの設定
processed_dir = Path(out_root) / "processed" / split
labels_dir = processed_dir / "labels"
images_dir = processed_dir / "images"
mapping_file = processed_dir / "image_mapping.json"

# ディレクトリの作成（失敗時はエラー）
try:
    images_dir.mkdir(parents=True, exist_ok=True)
    # 作成されたパスを確認（MyDrive直下でないことを再確認）
    if str(images_dir.parent.parent.parent) == "/content/drive/MyDrive":
        raise ValueError(
            f"❌ エラー: 出力先がMyDrive直下になってしまいます: {images_dir}\n"
            f"   OUT_ROOTを確認してください: {out_root}"
        )
except OSError as e:
    raise OSError(
        f"❌ エラー: 出力ディレクトリの作成に失敗しました: {images_dir}\n"
        f"   エラー詳細: {e}\n"
        f"   OUT_ROOTを確認してください: {out_root}"
    )

print("=" * 80)
print("パス確認")
print("=" * 80)
print(f"ラベルディレクトリ: {labels_dir}")
print(f"  存在確認: {labels_dir.exists()}")
print(f"画像ディレクトリ: {images_dir}")
print(f"  存在確認: {images_dir.exists()}")
print(f"マッピングファイル: {mapping_file}")
print(f"  存在確認: {mapping_file.exists()}")

# ラベルファイルの確認
label_files = list(labels_dir.glob('*.txt'))
print(f"\n既存のラベルファイル数: {len(label_files)} 件")

if not mapping_file.exists():
    print(f"\n⚠️  エラー: マッピングファイルが見つかりません: {mapping_file}")
    print("  先に preprocess_kaggle_data.ipynb を実行してください。")
elif len(label_files) == 0:
    print(f"\n⚠️  エラー: ラベルファイルが見つかりません: {labels_dir}")
    print("  先に preprocess_kaggle_data.ipynb を実行してください。")
else:
    print("\n✓ 準備完了")


In [ ]:
# 画像マッピングファイルの読み込み
if not mapping_file.exists():
    print("⚠️  マッピングファイルが見つかりません。処理をスキップします。")
else:
    with open(mapping_file, 'r', encoding='utf-8') as f:
        image_mapping = json.load(f)
    
    print(f"読み込んだマッピング数: {len(image_mapping)} 件")
    
    # ラベルファイルのUUIDを取得
    label_uuids = {f.stem for f in label_files}
    
    # マッピングとラベルファイルの対応確認
    mapped_uuids = set(image_mapping.keys())
    
    print(f"\nラベルファイルのUUID数: {len(label_uuids)}")
    print(f"マッピングのUUID数: {len(mapped_uuids)}")
    
    # 共通のUUID
    common_uuids = label_uuids & mapped_uuids
    print(f"共通のUUID数: {len(common_uuids)}")
    
    # ラベルファイルのみ存在（画像が見つからない）
    labels_only = label_uuids - mapped_uuids
    if len(labels_only) > 0:
        print(f"\n⚠️  警告: ラベルファイルのみ存在（画像が見つからない）: {len(labels_only)} 件")
        if len(labels_only) <= 10:
            for uuid in list(labels_only)[:10]:
                print(f"  - {uuid}")
    
    # マッピングのみ存在（ラベルファイルがない）
    mapping_only = mapped_uuids - label_uuids
    if len(mapping_only) > 0:
        print(f"\n⚠️  警告: マッピングのみ存在（ラベルファイルがない）: {len(mapping_only)} 件")
    
    print(f"\n処理対象: {len(common_uuids)} 件の画像をコピーします")


In [ ]:
# 画像ファイルのコピー
if not mapping_file.exists():
    print("⚠️  マッピングファイルが見つかりません。処理をスキップします。")
else:
    copied_count = 0
    skipped_count = 0
    error_count = 0
    
    # 共通のUUIDのみ処理
    for uuid in tqdm(common_uuids, desc="画像をコピー中"):
        source_image_path = Path(image_mapping[uuid])
        
        # 元の拡張子を取得
        ext = source_image_path.suffix.lower()
        if ext not in ['.png', '.jpg', '.jpeg']:
            # 拡張子がない場合は.pngを試す
            ext = '.png'
        
        target_image_path = images_dir / f"{uuid}{ext}"
        
        # 既に存在する場合はスキップ
        if target_image_path.exists():
            skipped_count += 1
            continue
        
        # ソース画像が存在するか確認
        if not source_image_path.exists():
            error_count += 1
            continue
        
        # 画像をコピー
        try:
            shutil.copy2(source_image_path, target_image_path)
            copied_count += 1
        except Exception as e:
            print(f"\n⚠️  エラー: {uuid} のコピーに失敗: {e}")
            error_count += 1
    
    print("\n" + "=" * 80)
    print("画像コピー完了")
    print("=" * 80)
    print(f"コピー成功: {copied_count} 件")
    print(f"スキップ（既存）: {skipped_count} 件")
    print(f"エラー: {error_count} 件")
    print(f"\n出力先: {images_dir}")
    
    # コピーされた画像ファイルの確認
    copied_images = list(images_dir.glob('*.*'))
    print(f"\nコピーされた画像ファイル数: {len(copied_images)} 件")
    
    # 1対1対応の確認
    image_uuids = {f.stem for f in copied_images}
    label_uuids_set = {f.stem for f in label_files}
    
    matched = image_uuids & label_uuids_set
    print(f"\n1対1対応確認:")
    print(f"  画像ファイル数: {len(image_uuids)}")
    print(f"  ラベルファイル数: {len(label_uuids_set)}")
    print(f"  対応しているペア数: {len(matched)}")
    
    if len(matched) == len(image_uuids) == len(label_uuids_set):
        print("\n✓ 完璧！全ての画像とラベルが1対1で対応しています。")
    else:
        print(f"\n⚠️  警告: 一部のファイルが対応していません")
        if len(image_uuids) != len(label_uuids_set):
            print(f"  画像のみ: {len(image_uuids - label_uuids_set)} 件")
            print(f"  ラベルのみ: {len(label_uuids_set - image_uuids)} 件")

## 画像とラベルの可視化（検証）

ランダムにいくつかのサンプルを選んで、画像とラベル（クラス、bbox）を表示します。

In [ ]:
# 画像とラベルの可視化
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random

# 表示するサンプル数
num_samples = 5

# 対応しているペアからランダムに選択
if len(matched) == 0:
    print("⚠️  表示できるサンプルがありません")
else:
    sample_uuids = random.sample(list(matched), min(num_samples, len(matched)))
    
    fig, axes = plt.subplots(num_samples, 1, figsize=(12, 4 * num_samples))
    if num_samples == 1:
        axes = [axes]
    
    for idx, uuid in enumerate(sample_uuids):
        # 画像を読み込み
        image_files = list(images_dir.glob(f"{uuid}.*"))
        if len(image_files) == 0:
            continue
        
        image_path = image_files[0]
        img = Image.open(image_path)
        img_width, img_height = img.size
        
        # ラベルを読み込み
        label_path = labels_dir / f"{uuid}.txt"
        labels = []
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        center_x = float(parts[1])
                        center_y = float(parts[2])
                        width = float(parts[3])
                        height = float(parts[4])
                        
                        # YOLO形式からピクセル座標に変換
                        x_center = center_x * img_width
                        y_center = center_y * img_height
                        bbox_width = width * img_width
                        bbox_height = height * img_height
                        
                        xmin = x_center - bbox_width / 2
                        ymin = y_center - bbox_height / 2
                        xmax = x_center + bbox_width / 2
                        ymax = y_center + bbox_height / 2
                        
                        labels.append({
                            'class_id': class_id,
                            'class_name': CLASSES[class_id] if class_id < len(CLASSES) else 'unknown',
                            'bbox': (xmin, ymin, xmax, ymax)
                        })
        
        # 画像を表示
        ax = axes[idx]
        ax.imshow(img)
        ax.axis('off')
        
        # バウンディングボックスを描画
        colors = plt.cm.tab20(range(len(CLASSES)))
        for label in labels:
            class_id = label['class_id']
            class_name = label['class_name']
            xmin, ymin, xmax, ymax = label['bbox']
            
            # バウンディングボックス
            rect = patches.Rectangle(
                (xmin, ymin), xmax - xmin, ymax - ymin,
                linewidth=2, edgecolor=colors[class_id % len(colors)],
                facecolor='none'
            )
            ax.add_patch(rect)
            
            # クラス名を表示
            ax.text(
                xmin, ymin - 5, class_name,
                color=colors[class_id % len(colors)],
                fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
            )
        
        # タイトル
        title = f"UUID: {uuid[:8]}... | 検出数: {len(labels)}"
        if len(labels) > 0:
            classes_found = [l['class_name'] for l in labels]
            title += f" | クラス: {', '.join(set(classes_found))}"
        ax.set_title(title, fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n表示したサンプル数: {len(sample_uuids)} 件")

In [ ]:
# 詳細な統計情報の表示
print("=" * 80)
print("データセット統計")
print("=" * 80)

print(f"\nディレクトリ構造:")
print(f"  {processed_dir}")
print(f"    ├── images/ ({len(list(images_dir.glob('*.*')))} ファイル)")
print(f"    └── labels/ ({len(list(labels_dir.glob('*.txt')))} ファイル)")

print(f"\n1対1対応:")
print(f"  画像ファイル: {len(image_uuids)} 件")
print(f"  ラベルファイル: {len(label_uuids_set)} 件")
print(f"  対応ペア: {len(matched)} 件")

# クラス別統計
from collections import defaultdict
class_stats = defaultdict(int)

for label_file in label_files:
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                if 0 <= class_id < len(CLASSES):
                    class_stats[CLASSES[class_id]] += 1

print(f"\nクラス別インスタンス数:")
for cls in CLASSES:
    count = class_stats[cls]
    print(f"  {cls:>3}: {count:>8} インスタンス")

total_instances = sum(class_stats.values())
print(f"\n総インスタンス数: {total_instances}")

print(f"\n✓ YOLO形式のデータセットが完成しました！")
print(f"  学習時は以下のパスを使用してください:")
print(f"    images: {images_dir}")
print(f"    labels: {labels_dir}")